<a href="https://colab.research.google.com/github/mayhd3/NSF-REU-2021/blob/main/AMImodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import os
import pandas as pd

if not os.path.exists('full.zip'):
  !curl --remote-name -H 'Accept: application/vnd.github.v3.raw' --location 'https://github.com/mayhd3/NSF-REU-2021/raw/main/full.zip'

full = pd.read_csv('full.zip')
full.columns = range(len(full.columns))

# group dataset by customer
groups = full.groupby(2)
meters = [groups.get_group(group) for group in groups.groups]

In [57]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPool2D, Dropout, Flatten
from keras.utils import np_utils
from scipy.signal import find_peaks
from keras.optimizers import Adam

# partition training and testing datasets
def bisect_ratio(array, ratio):
  return (
    [array[i] for i in range(len(array)) if i%ratio == 0],
    [array[i] for i in range(len(array)) if i%ratio != 0]
  )

def bhwc(twod):
  return np.expand_dims(np.array(twod), axis=-1).astype('float32')

X = [meter.iloc[:,5:].to_numpy().flatten() for meter in meters]
y = [np_utils.to_categorical([min(meter.iloc[0,0],1)], num_classes=2)[0] for meter in meters]

X_test, X_train = (bhwc(x) for x in bisect_ratio(X, 3))
y_test, y_train = (np.array(yb) for yb in bisect_ratio(y, 3))

print(y_train)

model = Sequential()
model.add(Conv1D(32, kernel_size=7, activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))    
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=52, verbose=2, batch_size=128,validation_split=0.3)
    
print(model.evaluate(X_test, y_test))


[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 

In [51]:
# cnn input is a 2D array of semi hourly consumption by week
X0 = [meter.iloc[:,5:].to_numpy() for meter in meters]
y0 = [
  {0: [0,0], 1: [0,1], 2:[1,0], 3:[1,1]}[meter.iloc[0,0]]
    for meter in meters
]

# fnn input is a 1D array of consumption over the year
X1 = [x.flatten() for x in X0]
y1 = [meter[1].to_numpy() for meter in meters]

# rnn input is a time series between consumption minima and maxima
X2 = [np.diff(find_peaks(np.mean(x, axis=1))[0]) for x in X0]
y2 = [y != 0 for y in y0]

X0_test, X0_train = bisect_ratio(X0, 3) # (bhwc(X) for X in )
X0_test = bhwc(X0_test)
X0_train = bhwc(X0_train)
y0_test, y0_train = (np.array(y) for y in bisect_ratio(y0, 3))

# cnn structure is (conv -> pool) x3 -> dropout -> dense x2
cnn = Sequential()
cnn.add(Conv2D(32, (3,3), input_shape=X0_train.shape[1:], activation='relu'))
#cnn.add(Conv2D(32,(4,4),input_shape=X0_train.shape, activation='relu'))
#cnn.add(MaxPool2D(pool_size=2))
#cnn.add(Conv2D(64,(26,26),activation='relu'))
#cnn.add(MaxPool2D(pool_size=2))
#cnn.add(Conv2D(128,(9,9),activation='relu'))
#cnn.add(MaxPool2D(pool_size=2))
#cnn.add(Dropout(0.0625))
cnn.add(Flatten())
#cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(2, activation='softmax'))

cnn.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

cnn.fit(
    X0_train,
    y0_train,
    epochs=52,
    batch_size=38
)

cnn.evaluate(X0_test, y0_test, verbose=2)

Epoch 1/52
4/4 [==============================] - 2s 326ms/step - loss: 267.4302 - accuracy: 0.4456
Epoch 2/52
4/4 [==============================] - 1s 317ms/step - loss: 310.9849 - accuracy: 0.3544
Epoch 3/52
4/4 [==============================] - 1s 321ms/step - loss: 294.1811 - accuracy: 0.8009
Epoch 4/52
4/4 [==============================] - 1s 314ms/step - loss: 195.1033 - accuracy: 0.3509
Epoch 5/52
4/4 [==============================] - 1s 319ms/step - loss: 355.2234 - accuracy: 0.2272
Epoch 6/52
4/4 [==============================] - 1s 326ms/step - loss: 106.7092 - accuracy: 0.4772
Epoch 7/52
4/4 [==============================] - 1s 324ms/step - loss: 94.2746 - accuracy: 0.4965
Epoch 8/52
4/4 [==============================] - 1s 322ms/step - loss: 66.4393 - accuracy: 0.6105
Epoch 9/52
4/4 [==============================] - 1s 320ms/step - loss: 54.2273 - accuracy: 0.3833
Epoch 10/52
4/4 [==============================] - 1s 316ms/step - loss: 32.0900 - accuracy: 0.4614
Epo

[4016.30517578125, 0.8815789222717285]